# Aerosol Mathematics Lab: Lognormal Size Distributions, Moments, and PM Metrics

## Learning goals
By the end of this notebook, you should be able to:
1) Represent an aerosol population using a lognormal size distribution.
2) Correctly work in **log-diameter space** (dN/dlnD) and integrate numerically.
3) Compute aerosol **moments** (M0, M1, M2, M3) and interpret physical meaning.
4) Convert a size distribution into **mass concentration** and estimate **PM1, PM2.5, PM10**.
5) Perform basic quality control: conservation (M0 ≈ N) and numerical convergence.

## Deliverables (submit)
- Plots requested in the notebook
- Reported numerical values (N check, moments, PM metrics)
- A short interpretation paragraph (150-250 words)


- Correct implementation of dN/dlnD and integration
- Moments + physical interpretation
- PM1/PM2.5/PM10 + units correct
- Plots are correct, labelled, readable
- QC checks: conservation + convergence


In [2]:
import numpy as np
import matplotlib.pyplot as plt


## Concept: Diameter space vs log-diameter space

We will work with the distribution:

\[
\frac{dN}{d\ln D}
\]

This is preferred for aerosols because size spans orders of magnitude (nm -> µm).

Key identity:
- If you have \(dN/d\ln D\), then
\[
dN = \left(\frac{dN}{d\ln D}\right) d\ln D
\]
- Relationship to \(dN/dD\):
\[
\frac{dN}{dD} = \frac{1}{D}\frac{dN}{d\ln D}
\]

**Task (answer in 2-3 lines):**
Why is using an evenly-spaced grid in \(\ln D\) numerically more stable than evenly spacing in \(D\)?

An evenly-spaced grid in ln D is numerically more stable because it allocates more resolution to the smaller diameters, where the distribution often changes most rapidly. This is crucial for accurately capturing the details of aerosol size distributions, which typically span several orders of magnitude. When using an evenly-spaced grid in D, larger diameters would consume a disproportionate number of grid points, leading to inefficient calculation and potentially inaccurate representation of the finer details at smaller sizes.

In [3]:
# ----------------------------
# Unit helpers
# ----------------------------
nm = 1e-9
um = 1e-6

def cm3_to_m3_factor():
    # 1 cm^3 = 1e-6 m^3, so (per cm^3) = (per m^3) * 1e-6 -> multiply by 1e6 to convert cm^-3 to m^-3
    return 1e6

# ----------------------------
# Base aerosol mode parameters (edit these)
# ----------------------------
N_cm3   = 5000.0      # total number concentration [cm^-3]
Dg      = 80.0 * nm   # geometric mean diameter [m]
sigmag  = 1.70        # geometric standard deviation [-]

# Material density (assume spherical particles)
rho = 1600.0          # [kg/m^3] typical for many aerosol mixtures (illustrative)

N = N_cm3 * cm3_to_m3_factor()  # convert to [m^-3]
N


5000000000.0

## Lognormal number distribution in log-diameter space

We use the canonical form:

\[
\frac{dN}{d\ln D} =
\frac{N}{\sqrt{2\pi}\ln\sigma_g}\,
\exp\left[-\frac{\left(\ln D-\ln D_g\right)^2}{2\left(\ln\sigma_g\right)^2}\right]
\]

Where:
- \(N\) is total number concentration [m⁻³]
- \(D_g\) is geometric mean diameter [m]
- \(\sigma_g\) is geometric standard deviation [-]

**Task:** Implement `dNdlnD(D, N, Dg, sigmag)` returning units of [m⁻³] per (unit lnD).


In [ ]:
def dNdlnD(D, N, Dg, sigmag):
    """
    Lognormal number size distribution in log-diameter space.

    Parameters
    ----------
    D : array-like
        Diameter [m]
    N : float
        Total number concentration [m^-3]
    Dg : float
        Geometric mean diameter [m]
    sigmag : float
        Geometric standard deviation [-]

    Returns
    -------
    dN_dlnD : array-like
        Number distribution dN/dlnD [m^-3]
    """
    lnsg = np.log(sigmag)
    pref = N / (np.sqrt(2*np.pi) * lnsg)
    exponent = - (np.log(D) - np.log(Dg))**2 / (2 * lnsg**2)
    return pref * np.exp(exponent)

# quick sanity check (should be finite)
print(dNdlnD(np.array([Dg]), N, Dg, sigmag))


In [ ]:
# Diameter grid (log-spaced)
Dmin = 3.0 * nm
Dmax = 10.0 * um
npts = 2000

D = np.logspace(np.log10(Dmin), np.log10(Dmax), npts)
lnD = np.log(D)

# For integration in lnD space, compute dlnD
dlnD = np.diff(lnD)  # length npts-1
dlnD_mean = np.mean(dlnD)

print("Grid summary:")
print("D range [m]:", D[0], "to", D[-1])
print("Mean dlnD:", dlnD_mean)


In [ ]:
dist = dNdlnD(D, N, Dg, sigmag)

plt.figure()
plt.semilogx(D / nm, dist)
plt.xlabel("Diameter D [nm]")
plt.ylabel("dN/dlnD [m$^{-3}$]")
plt.title("Lognormal aerosol number distribution (single mode)")
plt.grid(True, which="both", ls="--", alpha=0.4)
plt.show()


## QC Check 1: Conservation of number

Compute:

\[
N_{\text{num}} = \int \frac{dN}{d\ln D}\, d\ln D
\]

This should match your input \(N\) (within a small numerical tolerance).


In [ ]:
# Numerical integration in lnD space using trapezoidal rule:
# trapz(y, x) expects x array same length as y
N_num = np.trapz(dist, lnD)

rel_err = (N_num - N) / N

print(f"N input    = {N:.6e} m^-3")
print(f"N numeric  = {N_num:.6e} m^-3")
print(f"Rel. error = {rel_err:.3e}")


## Moments of the distribution

Define moments in lnD-space:

\[
M_k = \int D^k \frac{dN}{d\ln D}\, d\ln D
\]

Compute \(M_0, M_1, M_2, M_3\).

Interpretation (you will write in your submission):
- \(M_0\): number concentration
- \(M_2\): related to surface area (chemistry/heterogeneous reactions)
- \(M_3\): related to volume and mass (PM metrics)


In [ ]:
def moment_k(D, dN_dlnD, lnD, k):
    return np.trapz((D**k) * dN_dlnD, lnD)

M0 = moment_k(D, dist, lnD, 0)
M1 = moment_k(D, dist, lnD, 1)
M2 = moment_k(D, dist, lnD, 2)
M3 = moment_k(D, dist, lnD, 3)

print("Moments (SI units):")
print(f"M0 = {M0:.6e}  [m^-3]")
print(f"M1 = {M1:.6e}  [m^-2]")
print(f"M2 = {M2:.6e}  [m^-1]")
print(f"M3 = {M3:.6e}  [m^0] (dimensionless volume-per-volume scaling before constants)")


## Convert moments to aerosol bulk properties (assuming spheres)

Surface area concentration:
\[
S = \int \pi D^2 \frac{dN}{d\ln D} d\ln D = \pi M_2
\]
Units: m^2 per m^3 = m^-^1

Volume concentration:
\[
V_{\text{tot}} = \int \frac{\pi}{6} D^3 \frac{dN}{d\ln D} d\ln D = \frac{\pi}{6} M_3
\]
Units: m^3 per m^3 (dimensionless)

Mass concentration:
\[
C_m = \rho\,V_{\text{tot}}
\]
Units: kg/m^3, convert to µg/m^3.


In [ ]:
S = np.pi * M2                 # [m^-1]
Vtot = (np.pi/6.0) * M3        # [-] (m^3/m^3)
Cm_kg_m3 = rho * Vtot          # [kg/m^3]
Cm_ug_m3 = Cm_kg_m3 * 1e9      # [µg/m^3] because 1 kg = 1e9 µg

print(f"Surface area conc S      = {S:.6e}  [m^-1]")
print(f"Volume conc Vtot         = {Vtot:.6e}  [m^3/m^3]")
print(f"Mass conc Cm             = {Cm_ug_m3:.3f} [µg/m^3]")


## PM metrics: PM1, PM2.5, PM10

Compute mass integrated up to a cutoff diameter \(D_c\):

\[
PM(D_c) = \rho \int_0^{D_c} \frac{\pi}{6} D^3 \frac{dN}{d\ln D} d\ln D
\]

We will evaluate:
- \(D_c = 1 \,\mu m\)  → PM1
- \(D_c = 2.5 \,\mu m\) → PM2.5
- \(D_c = 10 \,\mu m\) → PM10


In [ ]:
def pm_cutoff(D, dN_dlnD, lnD, Dc, rho):
    mask = D <= Dc
    Vpart = np.trapz((np.pi/6.0) * (D[mask]**3) * dN_dlnD[mask], lnD[mask])
    Cm = rho * Vpart                # kg/m^3
    return Cm * 1e9                 # µg/m^3

PM1   = pm_cutoff(D, dist, lnD, 1.0*um,  rho)
PM25  = pm_cutoff(D, dist, lnD, 2.5*um,  rho)
PM10  = pm_cutoff(D, dist, lnD, 10.0*um, rho)

print(f"PM1   = {PM1:.3f}  µg/m^3")
print(f"PM2.5 = {PM25:.3f} µg/m^3")
print(f"PM10  = {PM10:.3f} µg/m^3")


In [ ]:
# cumulative mass as a function of diameter (use running trapezoid in lnD space)
mass_integrand = rho * (np.pi/6.0) * (D**3) * dist   # kg/m^3 per lnD
cum_mass = np.cumsum(0.5*(mass_integrand[1:]+mass_integrand[:-1]) * np.diff(lnD))  # kg/m^3
cum_mass_ug_m3 = cum_mass * 1e9

plt.figure()
plt.semilogx(D[1:]/um, cum_mass_ug_m3)
plt.axvline(1.0,  ls="--")
plt.axvline(2.5,  ls="--")
plt.axvline(10.0, ls="--")
plt.xlabel("Diameter D [µm]")
plt.ylabel("Cumulative mass [µg/m$^3$]")
plt.title("Cumulative mass vs diameter")
plt.grid(True, which="both", ls="--", alpha=0.4)
plt.show()


## Add a second mode (bimodal aerosol)

Add a coarse mode (e.g., sea salt / dust-like) and compare PM metrics.

Suggested coarse mode:
- \(N_2\) ~ 5–50 cm⁻³
- \(D_{g2}\) ~ 2 µm
- \(\sigma_{g2}\) ~ 2.0

**Task:**
1) Build the total distribution as the sum of two lognormals.
2) Plot fine, coarse, and total dN/dlnD.
3) Recompute PM1, PM2.5, PM10 and discuss which PM metric changes most and why.


In [ ]:
# Coarse mode parameters (editable)
N2_cm3  = 20.0
Dg2     = 2.0 * um
sigmag2 = 2.0

N2 = N2_cm3 * cm3_to_m3_factor()

dist1 = dNdlnD(D, N,  Dg,  sigmag)
dist2 = dNdlnD(D, N2, Dg2, sigmag2)
distT = dist1 + dist2

plt.figure()
plt.semilogx(D/um, dist1, label="Fine mode")
plt.semilogx(D/um, dist2, label="Coarse mode")
plt.semilogx(D/um, distT, label="Total", linewidth=2)
plt.xlabel("Diameter D [µm]")
plt.ylabel("dN/dlnD [m$^{-3}$]")
plt.title("Bimodal aerosol number distribution")
plt.grid(True, which="both", ls="--", alpha=0.4)
plt.legend()
plt.show()

PM1_T   = pm_cutoff(D, distT, lnD, 1.0*um,  rho)
PM25_T  = pm_cutoff(D, distT, lnD, 2.5*um,  rho)
PM10_T  = pm_cutoff(D, distT, lnD, 10.0*um, rho)

print(f"Bimodal PM1   = {PM1_T:.3f}  µg/m^3")
print(f"Bimodal PM2.5 = {PM25_T:.3f} µg/m^3")
print(f"Bimodal PM10  = {PM10_T:.3f} µg/m^3")


## QC Check 2: Numerical convergence with grid resolution

A stable numerical solution should not change much if you increase the number of grid points.

**Task:** Compute PM2.5 for npts = 200, 500, 2000 and compare.
Report the relative differences.


In [ ]:
def compute_PM25_for_npts(npts_test, use_bimodal=True):
    D_test = np.logspace(np.log10(Dmin), np.log10(Dmax), npts_test)
    lnD_test = np.log(D_test)

    dist_f = dNdlnD(D_test, N, Dg, sigmag)

    if use_bimodal:
        dist_c = dNdlnD(D_test, N2, Dg2, sigmag2)
        dist_total = dist_f + dist_c
    else:
        dist_total = dist_f

    return pm_cutoff(D_test, dist_total, lnD_test, 2.5*um, rho)

vals = {}
for n in [200, 500, 2000]:
    vals[n] = compute_PM25_for_npts(n, use_bimodal=True)

for n, v in vals.items():
    print(f"npts={n:4d} -> PM2.5 = {v:.6f} µg/m^3")

ref = vals[2000]
print("\nRelative difference vs npts=2000:")
for n in [200, 500]:
    print(f"npts={n:4d}: (PM2.5 - ref)/ref = {(vals[n]-ref)/ref:.3e}")


## Interpretation (150-250 words)

Write one paragraph addressing:
1) What physical aerosol type could the fine mode represent (secondary aerosol / urban / biomass burning) and why?
2) What does adding the coarse mode do to PM1 vs PM10 and why?
3) Which moment (M0, M2, M3) is most sensitive to coarse particles and why (use the D^k weighting idea)?


## Optional analytic check

For a lognormal defined in **dN/dlnD** form, the kth moment satisfies:

\[
M_k = N\,D_g^k \exp\left(\frac{1}{2} k^2 (\ln\sigma_g)^2\right)
\]

**Task:** Compare analytic vs numeric for the single fine mode for k = 0,1,2,3.


In [ ]:
def moment_k_analytic(N, Dg, sigmag, k):
    return N * (Dg**k) * np.exp(0.5 * (k**2) * (np.log(sigmag)**2))

print("Single-mode analytic vs numeric moments:")
for k, Mk_num in zip([0,1,2,3],[M0,M1,M2,M3]):
    Mk_an = moment_k_analytic(N, Dg, sigmag, k)
    print(f"k={k}: numeric={Mk_num:.6e}, analytic={Mk_an:.6e}, rel_err={(Mk_num-Mk_an)/Mk_an:.3e}")
